In [ ]:
"""
Filename:test.py
Author: Westbroobo
Date: 2024/12/13
"""
import pandas as pd

# 判断字符串中是否同时含有英文与数字
def check_string(s):
    has_letter = False
    has_digit = False
    for char in s:
        if char.isalpha():
            has_letter = True
        elif char.isdigit():
            has_digit = True
        if has_letter and has_digit:
            return True
    return False

# 数据修正
def vehicle_correction(vehicle):
    vehicle_split = vehicle.split(' ')
    if check_string(vehicle_split[-1]) and '-' in vehicle_split[-1]:
        year_split = vehicle_split[-1].split('-')
        length_year = len(year_split[-1])*2 + 1
        vehicle = vehicle[:-length_year] + ' ' + vehicle[-length_year:]
    elif check_string(vehicle_split[-1]) and '-' not in vehicle_split[-1]:
        str_end = vehicle_split[-1][-4:]
        if check_string(str_end):
            vehicle = vehicle[:-2] + ' ' + vehicle[-2:]
        else:
            vehicle = vehicle[:-4] + ' ' + vehicle[-4:]
    else:
        vehicle = vehicle
    return vehicle.replace('  ', ' ').replace(', ', ',').replace('（', '(').replace('）', ')')

# 数据标准化
def vehicle_normalization(vehicle):
    vehicle = vehicle_correction(vehicle)
    if ',' in vehicle:
        vehicle_split = vehicle.split(',')
        if ' ' in vehicle_split[-1]:
            make = vehicle_split[0].split(' ')[0]
            if make in ['Land', 'LAND', 'Alfa', 'ALFA', 'Aston', 'ASTON']:
                make = vehicle_split[0].split(' ')[0] + ' ' + vehicle_split[0].split(' ')[1]
            list_vehicle = [vehicle_split[0]]
            for v in vehicle_split[1:]:
                vehicle_ = make + ' ' + v
                list_vehicle.append(vehicle_)
            vehicle = '\n'.join(list_vehicle)
        else:
            make_model = ' '.join(vehicle_split[0].split(' ')[:-1])
            list_vehicle = [vehicle_split[0]]
            for v in vehicle_split[1:]:
                vehicle_ = make_model + ' ' + v
                list_vehicle.append(vehicle_)
            vehicle = '\n'.join(list_vehicle)
    return vehicle.replace('(', '').replace(')', '').strip()

# 年份判断
def year_(year):
    if len(year) == 2:
        if year in ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09'] or year in [str(i) for i in range(30)]:
            return '20' + year
        else:
            return '19' + year
    else:
        return year

# 年份标准化
def vehicle_handle_1(vehicle):
    if '-' not in vehicle.split(' ')[-1]:
        return year_(vehicle.split(' ')[-1])
    else:
        years = vehicle.split(' ')[-1].split('-')
        year_1 = years[-1]
        year_1 = year_(year_1)
        year_2 = years[0]
        year_2 = year_(year_2)
        year_min = str(min(int(year_1), int(year_2)))
        year_max = str(max(int(year_1), int(year_2)))
        year_result = year_min + '-' + year_max
        return year_result

# 车型标准化
def vehicle_handle_2(vehicle):
    try:
        vehicle_split = vehicle.split(' ')
        make_model = ' '.join(vehicle_split[:-1])
        years = vehicle_handle_1(vehicle)
        return make_model + ' ' + years
    except:
        return ''

df = pd.read_excel('./车型源数据.xlsx', dtype=str)
df['Vehicle'] = df['Vehicle'].apply(lambda x: x.split('\n'))
df = df.explode('Vehicle').fillna('')
df['Vehicle'] = df['Vehicle'].apply(lambda x:vehicle_normalization(x))

df['Vehicle'] = df['Vehicle'].apply(lambda x: x.split('\n'))
df = df.explode('Vehicle').fillna('')
df['Vehicle'] = df['Vehicle'].apply(lambda x:vehicle_handle_2(x))

df = df.groupby('No.', as_index=False, sort=False).agg({'Vehicle': list})
df['Vehicle'] = df['Vehicle'].apply(lambda x: '\n'.join(x))
df.to_excel('./车型标准化.xlsx', index=False)

